In [4]:
import tensorflow as tf
import gym
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from gym.envs.box2d import CarRacing
from stable_baselines.common import make_vec_env
from stable_baselines.common.policies import CnnPolicy
from stable_baselines.common.vec_env import DummyVecEnv,VecVideoRecorder
from stable_baselines import PPO2
import glob
import base64
import wandb
import time
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Any, Dict

## General Functions

In [2]:
from stable_baselines.common.callbacks import BaseCallback

class PPOCustomCallback(BaseCallback):
    """
    A custom callback that derives from ``BaseCallback``.

    :param verbose: (int) Verbosity level 0: not output 1: info 2: debug
    """
    def __init__(self, verbose=0):
        super(PPOCustomCallback, self).__init__(verbose)
        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseAlgorithm
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env = None  # type: Union[gym.Env, VecEnv, None]
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = None  # type: Dict[str, Any]
        # self.globals = None  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger = None  # stable_baselines3.common.logger
        # # Sometimes, for event callback, it is useful
        # # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]
        self.episodes = 0
        self.total_episode_reward = np.array([0,0,0,0], dtype=np.float32)

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        pass

    def _on_rollout_start(self) -> None:
        """
        A rollout is the collection of environment interaction
        using the current policy.
        This event is triggered before collecting new samples.
        """
        pass

    def _on_step(self) -> bool:
        # update commulative reward to log at the end of every episode
        self.total_episode_reward += self.locals["rewards"]
        # at the end of every episode       
        if np.array(self.locals["mb_dones"][-1]).any():
            # log the reward value if its time to not log 2 times
            if self.episodes % self.locals["log_interval"] != 0: 
                wandb.log({"reward_per_episode": np.mean(self.total_episode_reward[np.array(self.locals["mb_dones"][-1])==True]) })
            
            # if log interval has passed
            if self.episodes % self.locals["log_interval"] == 0:
                # Save your model and optimizer
                self.model.save(MODEL_SAVE_NAME)
                # Save as artifact for version control.
                artifact = wandb.Artifact(MODEL_SAVE_NAME, type='model')
                artifact.add_file(MODEL_SAVE_NAME+".zip")
                wandb.log_artifact(artifact)
                wandb.log({"reward_per_episode": np.mean(self.total_episode_reward[np.array(self.locals["mb_dones"][-1])==True]) })
            self.episodes += 1
            self.total_episode_reward[np.array(self.locals["mb_dones"][-1])==True] = 0

        
        return True

    def _on_rollout_end(self) -> None:
        """
        This event is triggered before updating the policy.
        """
        pass

    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the learn() method.
        """
        pass

# Baseline environment

## Training

In [2]:
env = lambda : CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_lanes_changes=1,
        num_tracks=1,
        allow_reverse=False,
        max_time_out=2,
        verbose=0,
        num_obstacles=0,
        max_step_reward=1
        )

In [4]:
num = 1

LOG_INTERVAL        = 100
WANDB_ID            = "" + str(num)
WNDB_NAME           = "Antoine" + str(num)
LOAD_SAVED_MODEL    = True
MODEL_SAVE_NAME     = "PPO_MODEL_" + str(num)
SAVED_MODEL_VERSION = "latest"

os.environ["WANDB_ENTITY"]  = "ant_ai"
os.environ["WANDB_PROJECT"] = "PPO_No_Obstacles"
os.environ["WANDB_RESUME"]  = "allow"
wandb.init(resume=WANDB_ID)
wandb.run.name = WNDB_NAME

env = make_vec_env(env, n_envs=4)

# Load model
if LOAD_SAVED_MODEL:
    try:
        model_artifact = wandb.use_artifact(MODEL_SAVE_NAME+':'+SAVED_MODEL_VERSION, type='model')
        artifact_dir = model_artifact.download()
        PPOmodel = PPO2.load(artifact_dir+"/"+MODEL_SAVE_NAME, env=env)
        print("LOAD SAVED PPΟ MODEL")

    except:
        print("NO MODEL FOUND")
else:
    if 'PPOmodel' not in globals():
        PPOmodel = PPO2(CnnPolicy, env, verbose=1)
        print("INITIALIZE NEW PPO MODEL")
    else:
        PPOmodel = PPO2.load(MODEL_SAVE_NAME, env=env)
        print("CONTINUE PPO MODEL TRAINING")

        # Train model
PPOmodel.learn(total_timesteps=2000000, log_interval=LOG_INTERVAL, callback=PPOCustomCallback())
PPOmodel.save(MODEL_SAVE_NAME)
env.close()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ant_ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



LOAD SAVED PPΟ MODEL
-------------------------------------
| approxkl           | 0.008186035  |
| clipfrac           | 0.104003906  |
| explained_variance | 0.761        |
| fps                | 71           |
| n_updates          | 1            |
| policy_entropy     | 0.54089534   |
| policy_loss        | -0.008199417 |
| serial_timesteps   | 128          |
| time_elapsed       | 0            |
| total_timesteps    | 512          |
| value_loss         | 0.6223811    |
-------------------------------------
--------------------------------------
| approxkl           | 0.004611495   |
| clipfrac           | 0.061523438   |
| ep_len_mean        | 541           |
| ep_reward_mean     | 54.6          |
| explained_variance | 0.947         |
| fps                | 87            |
| n_updates          | 100           |
| policy_entropy     | 0.6423588     |
| policy_loss        | -0.00

--------------------------------------
| approxkl           | 0.0070926268  |
| clipfrac           | 0.06640625    |
| ep_len_mean        | 581           |
| ep_reward_mean     | 55.2          |
| explained_variance | 0.941         |
| fps                | 85            |
| n_updates          | 1000          |
| policy_entropy     | 0.4912929     |
| policy_loss        | -0.0036474462 |
| serial_timesteps   | 128000        |
| time_elapsed       | 5.88e+03      |
| total_timesteps    | 512000        |
| value_loss         | 23.82411      |
--------------------------------------
--------------------------------------
| approxkl           | 0.0031172594  |
| clipfrac           | 0.04345703    |
| ep_len_mean        | 532           |
| ep_reward_mean     | 76.3          |
| explained_variance | 0.528         |
| fps                | 94            |
| n_updates          | 1100          |
| policy_entropy     | 0.43890023    |
| policy_loss        | -0.0025658493 |
| serial_timesteps   | 14

---------------------------------------
| approxkl           | 0.0018591783   |
| clipfrac           | 0.017089844    |
| ep_len_mean        | 486            |
| ep_reward_mean     | 89.4           |
| explained_variance | 0.578          |
| fps                | 87             |
| n_updates          | 2500           |
| policy_entropy     | 0.47421846     |
| policy_loss        | -0.00033091567 |
| serial_timesteps   | 320000         |
| time_elapsed       | 1.47e+04       |
| total_timesteps    | 1280000        |
| value_loss         | 102.04931      |
---------------------------------------
--------------------------------------
| approxkl           | 0.0045541925  |
| clipfrac           | 0.05810547    |
| ep_len_mean        | 488           |
| ep_reward_mean     | 65.9          |
| explained_variance | 0.633         |
| fps                | 87            |
| n_updates          | 2600          |
| policy_entropy     | 0.49139935    |
| policy_loss        | -0.0017980405 |
| serial_t

## Evaluate model

In [2]:
env = lambda : CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_lanes_changes=1,
        num_tracks=1,
        allow_reverse=False,
        max_time_out=2,
        verbose=0)
env = DummyVecEnv([env])

In [3]:
model = PPO2.load('Model Weights/PPO/Baseline Environment Agent/PPO_MODEL_1-v98/PPO_MODEL_1',env=env)

model.set_env(env)

for episode in range(10):
    obs = env.reset()
    total_reward = 0
    done=False
    while not done:
        action, states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        env.render()
    print(f'Episode: {episode}, Total Reward: {total_reward}')
env.close()

Episode: 0, Total Reward: [159.09991]
Episode: 1, Total Reward: [152.39993]
Episode: 2, Total Reward: [151.70003]
Episode: 3, Total Reward: [145.19997]
Episode: 4, Total Reward: [148.49901]
Episode: 5, Total Reward: [153.39998]
Episode: 6, Total Reward: [154.09999]
Episode: 7, Total Reward: [130.20055]
Episode: 8, Total Reward: [163.4997]
Episode: 9, Total Reward: [150.79999]


# Hard environment (Obstacles, Bonus, intersection, more complex tracks)

- Initial model is the one trained on the baseline environment.
- The idea is to fine tune it to work in a more complex environment 
- Bonus value = 50 vs Obstacle value = -50 (hard coded in environment code)

Complexity is set through 3 parameters: 
- num_tracks: 1 for basic track, 2 for more complex tracks with intersection (X, t type)
- num_lanes: number of lanes for the track 1 for normal, 2 for multiple lanes
- num_lanes_changes: number of time two lanes will be merged into 1 over the track

Obstacles/Bonus are controled by 2 parameters
- num_obstacles: number of obstacles by sub section of the track (if different than 0 then there are obstacles)
If the car touched an obstacle, 50 points are deducted from the score
- prop_good_obstacles: probability (set between 0-1) for an obstacles to be a bonus (yellow color instead of red + reward 50 points if touched)


## Training

In [5]:
env = lambda : CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_lanes_changes=1,
        num_tracks=2,
        allow_reverse=False,
        max_time_out=2,
        verbose=0,
        num_obstacles=5,
        prop_good_obstacles=0.5)

In [6]:
num = 1

LOG_INTERVAL        = 100
WANDB_ID            = "" + str(num)
WNDB_NAME           = "Antoine" + str(num)
LOAD_SAVED_MODEL    = False
MODEL_SAVE_NAME     = "PPO_MODEL_Obstacles" + str(num)
SAVED_MODEL_VERSION = "latest"

os.environ["WANDB_ENTITY"]  = "ant_ai"
os.environ["WANDB_PROJECT"] = "PPO_With_Obstacles"
os.environ["WANDB_RESUME"]  = "allow"
wandb.init(resume=WANDB_ID)
wandb.run.name = WNDB_NAME

env = make_vec_env(env, n_envs=4)

# Load model
if LOAD_SAVED_MODEL:
    try:
        model_artifact = wandb.use_artifact(MODEL_SAVE_NAME+':'+SAVED_MODEL_VERSION, type='model')
        artifact_dir = model_artifact.download()
        PPOmodel = PPO2.load(artifact_dir+"/"+MODEL_SAVE_NAME, env=env)
        print("LOAD SAVED PPΟ MODEL")

    except:
        print("NO MODEL FOUND")
else:
    if 'PPOmodel' not in globals():
        PPOmodel = PPO2.load('artifacts/PPO_MODEL_1-v98/PPO_MODEL_1',env=env)
        print("INITIALIZE NEW PPO MODEL")
    else:
        PPOmodel = PPO2.load(MODEL_SAVE_NAME, env=env)
        print("CONTINUE PPO MODEL TRAINING")

        # Train model
PPOmodel.learn(total_timesteps=5000000, log_interval=LOG_INTERVAL, callback=PPOCustomCallback())
PPOmodel.save(MODEL_SAVE_NAME)
env.close()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ant_ai. Use `wandb login --relogin` to force relogin


INITIALIZE NEW PPO MODEL
-------------------------------------
| approxkl           | 0.019688921  |
| clipfrac           | 0.1977539    |
| explained_variance | 0.249        |
| fps                | 50           |
| n_updates          | 1            |
| policy_entropy     | 0.68237      |
| policy_loss        | -0.009537595 |
| serial_timesteps   | 128          |
| time_elapsed       | 0            |
| total_timesteps    | 512          |
| value_loss         | 22.456802    |
-------------------------------------
---------------------------------------
| approxkl           | 0.005504113    |
| clipfrac           | 0.0703125      |
| ep_len_mean        | 1.1e+03        |
| ep_reward_mean     | 114            |
| explained_variance | 0.291          |
| fps                | 74             |
| n_updates          | 100            |
| policy_entropy     | 0.7449099      |
| policy_loss        | -0.00097121863 |
| serial_timesteps   | 12800          |
| time_elapsed       | 627            |
|

-------------------------------------
| approxkl           | 0.008492806  |
| clipfrac           | 0.10058594   |
| ep_len_mean        | 1.03e+03     |
| ep_reward_mean     | 142          |
| explained_variance | 0.00212      |
| fps                | 88           |
| n_updates          | 1500         |
| policy_entropy     | 0.75509036   |
| policy_loss        | -0.020744074 |
| serial_timesteps   | 192000       |
| time_elapsed       | 9.35e+03     |
| total_timesteps    | 768000       |
| value_loss         | 1.4775622    |
-------------------------------------
--------------------------------------
| approxkl           | 0.0059611653  |
| clipfrac           | 0.080078125   |
| ep_len_mean        | 979           |
| ep_reward_mean     | 137           |
| explained_variance | 0.112         |
| fps                | 79            |
| n_updates          | 1600          |
| policy_entropy     | 0.7214931     |
| policy_loss        | -0.0013934328 |
| serial_timesteps   | 204800        |
|

--------------------------------------
| approxkl           | 0.014655727   |
| clipfrac           | 0.07861328    |
| ep_len_mean        | 800           |
| ep_reward_mean     | 133           |
| explained_variance | 0.519         |
| fps                | 89            |
| n_updates          | 3000          |
| policy_entropy     | 0.49510506    |
| policy_loss        | -0.0076194443 |
| serial_timesteps   | 384000        |
| time_elapsed       | 1.87e+04      |
| total_timesteps    | 1536000       |
| value_loss         | 61.84205      |
--------------------------------------
-------------------------------------
| approxkl           | 0.0033818914 |
| clipfrac           | 0.03173828   |
| ep_len_mean        | 949          |
| ep_reward_mean     | 144          |
| explained_variance | 0.163        |
| fps                | 80           |
| n_updates          | 3100         |
| policy_entropy     | 0.79401845   |
| policy_loss        | -0.002059787 |
| serial_timesteps   | 396800      

--------------------------------------
| approxkl           | 0.004585153   |
| clipfrac           | 0.06298828    |
| ep_len_mean        | 856           |
| ep_reward_mean     | 135           |
| explained_variance | 0.703         |
| fps                | 87            |
| n_updates          | 4500          |
| policy_entropy     | 0.42151642    |
| policy_loss        | -0.0025164934 |
| serial_timesteps   | 576000        |
| time_elapsed       | 2.81e+04      |
| total_timesteps    | 2304000       |
| value_loss         | 16.7984       |
--------------------------------------
-------------------------------------
| approxkl           | 0.009070338  |
| clipfrac           | 0.06201172   |
| ep_len_mean        | 856          |
| ep_reward_mean     | 111          |
| explained_variance | 0.859        |
| fps                | 86           |
| n_updates          | 4600         |
| policy_entropy     | 0.46444973   |
| policy_loss        | -0.019557245 |
| serial_timesteps   | 588800      

--------------------------------------
| approxkl           | 0.006166261   |
| clipfrac           | 0.087402344   |
| ep_len_mean        | 836           |
| ep_reward_mean     | 149           |
| explained_variance | 0.647         |
| fps                | 88            |
| n_updates          | 6000          |
| policy_entropy     | 0.5455546     |
| policy_loss        | 0.00021869713 |
| serial_timesteps   | 768000        |
| time_elapsed       | 3.76e+04      |
| total_timesteps    | 3072000       |
| value_loss         | 25.266825     |
--------------------------------------
------------------------------------
| approxkl           | 0.007923445 |
| clipfrac           | 0.092285156 |
| ep_len_mean        | 831         |
| ep_reward_mean     | 150         |
| explained_variance | 0.834       |
| fps                | 85          |
| n_updates          | 6100        |
| policy_entropy     | 0.6249434   |
| policy_loss        | -0.01039305 |
| serial_timesteps   | 780800      |
| time_e

--------------------------------------
| approxkl           | 0.0039464696  |
| clipfrac           | 0.052734375   |
| ep_len_mean        | 816           |
| ep_reward_mean     | 143           |
| explained_variance | 0.685         |
| fps                | 71            |
| n_updates          | 7500          |
| policy_entropy     | 0.69118667    |
| policy_loss        | -0.0022888347 |
| serial_timesteps   | 960000        |
| time_elapsed       | 4.71e+04      |
| total_timesteps    | 3840000       |
| value_loss         | 76.38838      |
--------------------------------------
-------------------------------------
| approxkl           | 0.0040715868 |
| clipfrac           | 0.04736328   |
| ep_len_mean        | 890          |
| ep_reward_mean     | 153          |
| explained_variance | 0.742        |
| fps                | 88           |
| n_updates          | 7600         |
| policy_entropy     | 0.6715275    |
| policy_loss        | -0.011488377 |
| serial_timesteps   | 972800      

-------------------------------------
| approxkl           | 0.0044385414 |
| clipfrac           | 0.0546875    |
| ep_len_mean        | 799          |
| ep_reward_mean     | 159          |
| explained_variance | 0.255        |
| fps                | 76           |
| n_updates          | 9000         |
| policy_entropy     | 0.59396213   |
| policy_loss        | -0.007685321 |
| serial_timesteps   | 1152000      |
| time_elapsed       | 5.65e+04     |
| total_timesteps    | 4608000      |
| value_loss         | 6.30007      |
-------------------------------------
-------------------------------------
| approxkl           | 0.0070681768 |
| clipfrac           | 0.08642578   |
| ep_len_mean        | 820          |
| ep_reward_mean     | 175          |
| explained_variance | 0.832        |
| fps                | 87           |
| n_updates          | 9100         |
| policy_entropy     | 0.65974665   |
| policy_loss        | -0.008000364 |
| serial_timesteps   | 1164800      |
| time_elaps

In [7]:
wandb.finish()

reward_per_episode,▄▇▄▄▆▇▄▆▄▅▇▇▆▇▆▃▅▇▇▆▇▇▇▆▆█▆▁▂▆██▆▆█▅██▆▇
reward_per_episode,260.59921


## Evaluate model

In [5]:
env = lambda : CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_lanes_changes=1,
        num_tracks=2,
        allow_reverse=False,
        max_time_out=2,
        verbose=0,
        num_obstacles=10,
        prop_good_obstacles=0.5)
env = DummyVecEnv([env])

In [7]:
model = PPO2.load('Model Weights/PPO/Complex Environment Agent/PPO_MODEL_OBS-v56/PPO_MODEL_Obstacles1',env=env)

model.set_env(env)

test_set=[]
for episode in tqdm(range(100)):
    obs = env.reset()
    total_reward = 0
    done=False
    while not done:
        action, states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        env.render()
    print(f'Episode: {episode}, Total Reward: {total_reward}')
    test_set.append(total_reward)
env.close()

print('Average Reward: ',np.mean(test_set))

  1%|▊                                                                                 | 1/100 [00:17<29:15, 17.73s/it]

Episode: 0, Total Reward: [228.20038]


  2%|█▋                                                                                | 2/100 [00:29<23:29, 14.38s/it]

Episode: 1, Total Reward: [116.59984]


  3%|██▍                                                                               | 3/100 [00:41<21:38, 13.38s/it]

...

Episode: 97, Total Reward: [226.19849]


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [28:58<00:18, 18.25s/it]

Episode: 98, Total Reward: [312.8993]


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [29:13<00:00, 17.53s/it]

Episode: 99, Total Reward: [260.49854]
Average Reward:  198.06438


In [8]:
## Get summary statistic
pd.DataFrame(test_set).describe()

,0
count,100.000000
mean,198.064377
std,137.629547
min,-117.000015
25%,119.674997
50%,223.047943
75%,293.948242
max,476.595093


## Record Videos

In [3]:
'''
Complexity is set through 3 parameters: 
num_tracks: 1 for basic track, 2 for more complex tracks with intersection (X, t type)
num_lanes: number of lanes for the track 1 for normal, 2 for multiple lanes
num_lanes_changes: number of time two lanes will be merged into 1 over the track


Obstacles/Bonus are controled by 2 parameters
num_obstacles: number of obstacles by sub section of the track (if different than 0 then there are obstacles)
If the car touched an obstacle, 50 points are deducted from the score
prop_good_obstacles: probability (set between 0-1) for an obstacles to be a bonus (yellow color instead of red + reward 50 points if touched)
'''


env = lambda : CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_lanes_changes=1,
        num_tracks=1,
        allow_reverse=False,
        max_time_out=2,
        verbose=0,
        num_obstacles=0,
        prop_good_obstacles=0.1)
env = DummyVecEnv([env])



In [4]:
# model = PPO2.load('artifacts/PPO_MODEL_OBS-v56/PPO_MODEL_Obstacles1',env=env) #Tuned agent trained on complex track and obstacles/bonus
model = PPO2.load('artifacts/PPO_MODEL_1-v98/PPO_MODEL_1',env=env) # Baseline agent trained on the base environment

# Record the video starting at the first step
env = VecVideoRecorder(env, 'video/',
                       record_video_trigger=lambda x: x == 0, video_length=1500,
                       name_prefix="ppo_agent-based")

for episode in range(10):
    obs = env.reset()
    total_reward = 0
    done=False
    while not done:
        action, states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        env.render()
    print(f'Episode: {episode}, Total Reward: {total_reward}')
env.close()


Episode: 0, Total Reward: [135.80063]
Episode: 1, Total Reward: [128.30043]
Episode: 2, Total Reward: [150.4997]
Episode: 3, Total Reward: [140.50046]
Episode: 4, Total Reward: [-21.799583]
Episode: 5, Total Reward: [142.3002]
Episode: 6, Total Reward: [149.90007]
Episode: 7, Total Reward: [126.70076]
Episode: 8, Total Reward: [146.3001]
Episode: 9, Total Reward: [146.09993]
